## X-track plotting

### Imports

In [ ]:
# Import standard libraries
import numpy as np
from cpymad.madx import Madx
import json
import xtrack as xt
import xfields as xf
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout
from ipywidgets import interact
from plotly.subplots import make_subplots

# Import plotting functions
from plotting_functions import return_plot_lattice_with_tracking

# Nicer plots
pio.templates.default = "plotly_white"

# Different backend
import plotly.io as pio
# pio.renderers.default = "vscode"

# Color list [blue, red, green, purple, orange,light blue, pink, light green, light violet, yellow]
l_colors = px.colors.qualitative.Plotly




### Load json files, build trackers and corresponding dataframes

In [ ]:
# Load the lines into json files
with open("json_lines/line_b1.json", "r") as fid:
    dct_b1 = json.load(fid)
    line_b1 = xt.Line.from_dict(dct_b1)

with open("json_lines/line_b4.json", "r") as fid:
    dct_b4 = json.load(fid)
    line_b4 = xt.Line.from_dict(dct_b4)

# Build trackers
tracker_b1 = line_b1.build_tracker()
tracker_b4 = line_b4.build_tracker()




In [ ]:
from dash import Dash, html, dcc, Input, Output
from jupyter_dash import JupyterDash
import dash_cytoscape as cyto

app = JupyterDash(__name__)

In [63]:
nodes = []
edges = []
set_nodes = set()
set_edges = set()
# Browse all the vars
for key in tracker_b1.vars._value.keys():
    if key not in set_nodes:
        # convert key to var string
        key_str = "vars['"+key+"']"
        # Add the node
        nodes.append(
            {
                "data": {"id": key_str, "label": key_str},
            }
        )
        # Add node to the set 
        set_nodes.add(key_str)
    
    # Add all children
    for child in tracker_b1.vars[key]._find_dependant_targets():
        if str(child) not in set_nodes:
            # Add the node
            nodes.append(
                {
                    "data": {"id": str(child), "label": str(child)},
                }
            )
            # Add node to the set 
            set_nodes.add(str(child))
        if (key_str, str(child)) not in set_edges and key_str!=str(child):
            # Add the edge
            edges.append(
                {"data": {"source": key_str, "target": str(child)}}
            )
            # Add edge to the set
            set_edges.add((key_str, str(child)))
        

        # Add subsequent grandchildren
        for grandchild in tracker_b1.vars[str(child).split("'")[1]]._find_dependant_targets():
            if str(grandchild) not in set_nodes:
                # Add the node
                nodes.append(
                    {
                        "data": {"id": str(grandchild), "label": str(grandchild)},
                    }
                )
                # Add node to the set 
                set_nodes.add(str(grandchild))
            if (str(child), str(grandchild)) not in set_edges and str(grandchild)!=str(child):
                edges.append(
                    {"data": {"source": str(child), "target": str(grandchild)}}
                )




In [65]:
print(edges[-100:])

[{'data': {'source': "vars['l.bpmwg']", 'target': "element_refs['bpmwg.a5l3.b1'].length"}}, {'data': {'source': "element_refs['bpmwg.a5l3.b1'].length", 'target': "vars['bpmwg.a5l3.b1']"}}, {'data': {'source': "vars['l.bpmwg']", 'target': "element_refs['bpmwg.a5l3.b1']"}}, {'data': {'source': "element_refs['bpmwg.a5l3.b1']", 'target': "vars['bpmwg.a5l3.b1']"}}, {'data': {'source': "vars['l.tcapd']", 'target': "element_refs['tcapd.5l3.b1']"}}, {'data': {'source': "vars['l.tcapd']", 'target': "element_refs['tcapd.5l3.b1'].length"}}, {'data': {'source': "vars['l.tcapd']", 'target': "element_refs['tcapd.5l3.b1']"}}, {'data': {'source': "element_refs['tcapd.5l3.b1']", 'target': "vars['tcapd.5l3.b1']"}}, {'data': {'source': "vars['l.tcapd']", 'target': "element_refs['tcapd.5l3.b1'].length"}}, {'data': {'source': "element_refs['tcapd.5l3.b1'].length", 'target': "vars['tcapd.5l3.b1']"}}, {'data': {'source': "vars['kqt5.l3']", 'target': "element_refs['mqwb.5l3.b1..2']"}}, {'data': {'source': "va

In [ ]:
elements = nodes + edges


app.layout = html.Div(
    [
        dcc.Dropdown(
            id="dropdown-update-layout",
            value="grid",
            clearable=False,
            options=[
                {"label": name.capitalize(), "value": name}
                for name in ["grid", "random", "circle", "cose", "concentric"]
            ],
        ),
        cyto.Cytoscape(
            id="cytoscape-update-layout",
            layout={"name": "grid"},
            style={"width": "100%", "height": "450px"},
            elements=elements,
        ),
    ]
)


@app.callback(
    Output("cytoscape-update-layout", "layout"),
    Input("dropdown-update-layout", "value"),
)
def update_layout(layout):
    return {"name": layout, "animate": True}


if __name__ == "__main__":
    app.run_server(debug=True, port = 8054)